# Q learning based solution for [FrozenLake-v0](https://gym.openai.com/envs/FrozenLake-v0)

Mostly based on explanations in [mnemstudio tutorial](http://mnemstudio.org/path-finding-q-learning-tutorial.htm) and this wonderful RL series in [Medium](https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-0-q-learning-with-tables-and-neural-networks-d195264329d0).

### Uses Numpy
This is a very basic implementation. Does not perform very well. 

In [1]:
import gym
import numpy as np

## Load Open AI gym

In [2]:
env = gym.make('FrozenLake-v0')

[2017-05-21 15:47:51,668] Making new env: FrozenLake-v0


In [10]:
# Load Gym parameters
state_size = env.observation_space.n
action_size = env.action_space.n

#### Hyperparameters

In [1]:
q_reduction = 0.1
gamma_reduction = 0.9
num_episodes = 100000

# print the reward for episode every n episodes
print_every = 10000

#### Global Parameters

In [17]:
# Initialize Q array with zeros
Q = np.zeros([state_size, action_size])

#### Play algorithm


In [18]:
def play():
    total_reward = 0
    for i in range(num_episodes):
        #Reset environment and get start state
        state = env.reset()
        episode_reward = 0
        done = False
        count = 0
        # Algorithm works by taking a step at a time based on Q learning. Gym tells us at each step if we reached `done`
        # state. Done state means either we failed or reach goal state. But it could be possible that our algorithm
        # keeps going in circles and never reaches done state. We should break this.
        # In my approach I assume that if our step count is twice size of the board that means we are not getting anywhere.
        break_loop_at = state_size * action_size * 2
        while count < break_loop_at and not done:
            count += 1
            
            # Look at current Q table and pick the action with maximum reward greedily. But according to the requirements
            # given in Game, there could be wind (read as noise) which could sway our decision. So we simulate noise by
            # adding random values to all actions in this state and then picking the max action (greedy)
            action_with_noise = Q[state, :] + np.random.randn(1, action_size)*(1.0/(count+1))
            action = np.argmax(action_with_noise)
            # Make the step and get the next_state and reward from env
            next_state,reward,done,_ = env.step(action)

            # Update Q table.
            Q[state, action] = q_reduction * Q[state, action] + (1-q_reduction)*(reward + gamma_reduction * np.max(Q[next_state:]))
            
            # Upadate episode reward
            episode_reward += reward
            
            # Update the state
            state = next_state
       
        total_reward += episode_reward
        if i % print_every == 0:
            print "Episode: {}, Total Reward: {}".format(i, "{0:.4f}".format(total_reward/(i+1)))

    print "================== Reward ================"
    print "Total Reward : {} ".format("{0:.4f}".format(total_reward/num_episodes))
    print "================== Q ================="    
    print Q

### Start Playing

In [19]:
play()

Episode: 0, Total Reward: 0.0000
Episode: 10000, Total Reward: 0.0069
Episode: 20000, Total Reward: 0.0096
Episode: 30000, Total Reward: 0.0118
Episode: 40000, Total Reward: 0.0124
Episode: 50000, Total Reward: 0.0129
Episode: 60000, Total Reward: 0.0131
Episode: 70000, Total Reward: 0.0137
Episode: 80000, Total Reward: 0.0139
Episode: 90000, Total Reward: 0.0146
================== Reward ================
Total Reward : 0.0150 
================== Q =================
[[ 0.73374625  0.7337464   0.73374777  0.7337464 ]
 [ 0.7337464   0.73374634  0.73374849  0.73382034]
 [ 0.73382034  0.74101357  0.74100334  0.74190052]
 [ 0.          0.73588472  0.          0.74190052]
 [ 0.72712984  0.7277258   0.72713044  0.7337536 ]
 [ 0.          0.          0.          0.        ]
 [ 0.73601705  0.757301    0.          0.81527361]
 [ 0.          0.          0.          0.        ]
 [ 0.72736642  0.68914406  0.72857276  0.72766779]
 [ 0.73441483  0.66105629  0.68840136  0.        ]
 [ 0.74695701  0.  